### CREAMOS LAS FUNCIONES
#### FUNCIONES QUE ABRE EL ARCHIVO

In [ ]:
def abreArchivo(nombre):
    try: 
        fichero = (open(str(nombre)+'.txt', 'x')) 
        print('se ha CREADO el archivo') 
        fichero.close()
        print('Se ha CERRADO el archivo')
    except:
        print('Archivo ya existe') 
    try: 
        fichero = (open(str(nombre)+'.txt', 'a')) 
        print('Archivo abierto en modo APPEND') 
    except Exception as e:
        print('Se ha producido un Error al intentar ABRIR el archivo en modo APPEND\n', e)
    return fichero

### Función que crea un archivo con un parametro str que le da nombre y que deja el archivo abierto en modo Append.

### Try revisa que exista el archivo y sino lo crea.

### Intentamos crear el archivo y nos dá un mensaje de exito o si existe uno de error.

### Tras esto, lo abre en modo append y en caso positivo nos devuelve de nuevo el mensaje de exito, en caso de error nos devolverá el mensaje correspondiente

### Finalmente nos devuelve el fichero.

#### FUNCION QUE CIERRA ARCHIVO

In [ ]:
def cierraArchivo(nombre):
    try:
        nombre.close()
        print('Se ha CERRADO el archivo')
    except:
        print('Error')
### Pasa la variable entera para que la cierre o de mensaje de error



#### FUNCION QUE SACA ATRIBUTOS

In [ ]:
def saca_atributos(items):
    x = len(items)-1
    lista = []
    while x >= 0:
        lista.append(items[x]['attributes'])
        x = x - 1
    return lista

### Esta función extrae los atributos mientras el item sea menor o igual a 0

### Creamos un bucle que recorra todos los datos y creamos una lista vacia para introducir los datos solicitados en ella.

### Mientras nuestra variable(objeto) sea cierta añadirá a la lista los datos extraidos.

###Imprimimos la lista para mostrarla

#### FUNCION QUE SACA LOS DATOS GEOMETRICOS Y LOS INSERTA CON EL OBKECT ID

In [ ]:
def saca_geografia(items):
    x = len(items)-1
    diccionario = {}
    while x >= 0:
        a = items[x]['attributes']
        b = items[x]['geometry']
        diccionario['objectID'] = a['OBJECTID'] 
        diccionario['rings'] = b['rings']
        x = x - 1
    return diccionario

###Esta función saca la geografía creando un diccionario vacío e introduciendo los datos que estan dentro de 'rings'.

### A continuación nos devuelve los datos extraidos en un diccionario

#### IMPORTAMOS REQUEST PARA HACER LAS LLAMADAS

In [ ]:
import requests

#### VARIABLES PARA BUCLES Y LLAMADAS

In [2]:
i = 0
limite = 2
offset = 1
count = 1
### Las dos primeras variables son para controlar el while (controlan numero de veces que se repite el bucle) Las siguientes dos variables son para controlar el URL (controlan que pido en cada request)

### HACEMOS LA LLAMADA PARA EXTRAER LOS DATOS
#### PRIMERO DAMOS VALOR A CADA PARTE DE LA URL

In [1]:
inicio = 'http://mapas.igme.es/gis/rest/services/Cartografia_Geologica/IGME_MAGNA_50/MapServer/10/query?'
f ='f=json' 
where = '&where=(1%3D1)%20AND%20(1%3D1)' 
geometry = '&returnGeometry=true'
spatial = '&spatialRel=esriSpatialRelIntersects' 
outfields = '&outFields=*' 
orderby = '&orderByFields=OBJECTID%20ASC'
outSR = '&outSR=102100' 
resultOffset = '&resultOffset=1' 
resultRecord = '&resultRecordCount=2'

#### Explicación de las equivalencias
f = Formato en el que pasara los datos

where = Condicion para la consulta

geomtry = Si pasa la geometria o no

orderby = El orden de la consulta

resultOffset = Cuantas columnas desde el principio antes de empezar a imprimir

resultRecord = Cuantas columnas imprimir

#### Cantidad de columnas de la llamada
resultOffset = '&resultOffset=1'

El número 1 indica el comienzo de la consulta

resultRecord = '&resultRecordCount=2'

El número 2 indica la llamada a el número de columnas que escojamos



### Generamos el archivo

In [ ]:
BD_objetos = abreArchivo('BD_objetos')
BD_atributos = abreArchivo('BD_atributos')
BD_objGeograf = abreArchivo('BD_objGeograf')

Generamos los 3 ficheros que necesitamos con objetos, atributos y Geografía



### Extracción a fichero¶


In [ ]:
while i < limite:
    url = inicio+f+where+geometry+spatial+outfields+orderby+outSR+resultOffset+str(offset)+resultRecord+str(count)
    res = requests.get(url)
    data = res.json()
    items = data['features']
    
    BD_objetos.write(str(items)) 

    atributos = saca_atributos(items)
    BD_atributos.write(str(atributos))
                     
    geografia = saca_geografia(items)
    BD_objGeograf.write(str(geografia))
                     
    print(offset) 
    offset += 1 
    i += 1

#### Creamos un bucle que itere por la cantidad de archivos que hayamos especificado

1. Unimos todos los valores en una URL
2. Hacemos una llamada en formato JSON y pedimos los datos
3. Solicitamos los datos dentro de 'features'
4. Escribimos los datos en el archivo en formato string
5. Utilizamos la función que saca los atributos y los escribe igual que el anterior
6. Usamos la función geografía y hará el mismo proceso
7. Imprimimos un contador de control para log
8. Aumentamos el offset para pasar al proximo item
9. Finalmente aumentamos el control del bucle


### Cerramos los archivos

In [ ]:
cierraArchivo(BD_objetos)
cierraArchivo(BD_atributos)
cierraArchivo(BD_objGeograf)

## Seguidamente, vamos a usar la información geográfica de cada litología para representar los polígonos correspondientes sobre un mapa

In [ ]:
### En primer lugar, vamos a instalar las librerías necesarias. 
### folium para representar el mapa, y proj para transformar las coordenadas.

import folium
from pyproj import Proj, transform

### Llamamos a la API y obtenemos toda la info sobre un objeto de nuestra DB

In [ ]:
# SACO DATOS

import requests

inicio = 'http://mapas.igme.es/gis/rest/services/Cartografia_Geologica/IGME_MAGNA_50/MapServer/10/query?'
f ='f=json' #Formato en el que pasara los datos
where = '&where=(1%3D1)%20AND%20(1%3D1)' #condicion para la consulta
geometry = '&returnGeometry=true'#si pasa la geometria o no
spatial = '&spatialRel=esriSpatialRelIntersects' #AVERIGUAR
outfields = '&outFields=*' #AVERIGUAR
orderby = '&orderByFields=OBJECTID%20ASC' #como viene ordenada la consulta
outSR = '&outSR=102100' #AVERIGUAR
resultOffset = '&resultOffset=250500' #cuantas columnas desde el principio antes de empezar a imprimir
resultRecord = '&resultRecordCount=1' #cuantas columnas imprimir

url = inicio+f+where+geometry+spatial+outfields+orderby+outSR+resultOffset+resultRecord
    
res = requests.get(url)
data = res.json()
item = data['features']
item

### Podemos ver que la información obtenida (que anteriormente hemos reunido en los archivos finales BD_objetos, BD_atributos y BD_objGeograf) contiene 5 variables:
#### Hoja : El número de hoja en la clasificación MAGNA en la que se encuentra el objeto requerido
#### DLO : Descripción litológica. Descripción de la unidad rocosa correspondiente al objeto en cuestión
#### ObjectID: Identificador de tipo de objeto dentro de la API
#### ID: Identificador de DLO
#### geometry: siguiendo un modelo ESRI, cada objeto tiene un 'ring' asociado. Cada ring se compone de una lista indefinida de puntos que, unidos entre sí, marcan la limitación geográfica del objeto solicitado.
### A continuación, limpiamos los datos para poder trabajar con las geometrías en solitario

In [ ]:
# SEPARO LA LISTA DE RINGS

rings = item[0]['geometry']
target = rings['rings']
target

## Seguidamente, transformamos las coordenadas en un formato manejable. La API nos las suministra en pseudo-mercator, y las transformamos al modelo (latitud , longitud) que pueda leer folium.

In [ ]:
# RECORRO LA LISTA Y TRANSFORMO EL FORMATO DE LAS COORDENADAS

inProj = Proj(init='epsg:3857')
outProj = Proj(init='epsg:4326')
lista = target[0]
convertidas = []
a = 0

while a < len(lista):
    x1,y1 = lista[a]
    punto = transform(inProj,outProj,x1,y1)
    convertidas.append(punto)
    a += 1

convertidas

### Además, invertimos los datos para que las coordenadas tengan coherencia

In [ ]:
#INVERTIR TUPLAS

b = 0
coordenadas = []

while b < len(convertidas):
    punto1 = convertidas[b][1]
    punto2 = convertidas[b][0]
    entrar = (punto1, punto2)
    coordenadas.append(entrar)
    b += 1

coordenadas

### Finalmente, mandamos la lista de coordenadas a folium para poder representar (en rojo) el polígono correspondiente al objeto solicitado. Añadimos a mano un popup descriptivo de la zona en cuestión (más adelante lo relacionaremos directamente con la DLO).

In [ ]:
#PRUEBA DE DIBUJO
m = folium.Map(location = coordenadas[0], zoom_start=16)

folium.Polygon(coordenadas, color="red", weight=2.5, fill_color='red', opacity=10, popup = 'Arcillas y limos. Zona endorreica').add_to(m)

m

# ANEXOS

## INTENTO DE WEBSCRAPING

### Importamos los paquetes


In [ ]:
import requests
from bs4 import BeautifulSoup

### Creamos un bucle para extraer la información de los PDF online¶


In [ ]:
for numero in [1,2]:
    URL = 'http://info.igme.es/cartografiadigital/datos/magna50/pdfs/d0_G50/Magna50_'+str(numero)+'.pdf'
    page = requests.get(URL)
    #page.enconding()
    soup = BeautifulSoup(page.content,'html.parser')#.decode('utf-8','WinAnsiEncoding')
    print(page.content)
    
soup = BeautifulSoup(page.content,'html.parser')
hojas = soup.find_all('td', class_='EnlaceMini')
hojas